Creating Triples and merging them for each relation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import json
import numpy as np

Load the "adf_with_tfidf.csv"

In [ ]:
# adf = pd.read_csv('/content/drive/MyDrive/IIITB Docs/sem2/EduEmbed/EduEmbedd/data_prepare/adf_pc_1.csv')
adf = pd.read_csv('/content/drive/MyDrive/IIITB Docs/sem2/EduEmbed/EduEmbedd/data_prepare/adf_with_tfidf.csv')

In [ ]:
adf

Below code creates a DataFrame adf1 which will contain the triples for the "l_text_topics" relation along with LDA probability as its weights

In [ ]:
adf1 = adf[['file_name','l_text_topics']]
adf1 = adf1.dropna()
adf1 = adf1.melt('file_name')
#df15 = df15.drop_duplicates(subset=['topics','variable','value'], keep='last')
adf1.rename(columns = {"file_name": "head"},  
          inplace = True) 
adf1.head(5)
print(adf1.shape)

adf1['prob'] = adf[['l_text_prob']]
adf1.head(50)

In [ ]:
adf1['value'] = adf1['value'].apply(lambda x:json.loads(x))
adf1['prob'] = adf1['prob'].apply(lambda x:json.loads(x))
type(adf1['prob'][0])
adf1 = adf1.explode(['value', 'prob'])
adf1['value'] = 'topic_' + adf1['value'].astype(str)
adf1.head(50)


In [ ]:
adf1.shape

Below code creates a DataFrame adf2 which will contain the triples for the "concept_vocab_index" relation along with Tf-Idf score as its weights

In [ ]:
adf2 = adf[['file_name','concept_vocab_index']]
adf2 = adf2.dropna()
adf2 = adf2.melt('file_name')
#df15 = df15.drop_duplicates(subset=['topics','variable','value'], keep='last')
adf2.rename(columns = {"file_name": "head"},  
          inplace = True) 
lst_col = 'value'
# adf2 = pd.DataFrame({
#     col:np.repeat(adf2[col].values, adf2[lst_col].str.len())
#     for col in adf2.columns.difference([lst_col])
#     }).assign(**{lst_col:np.concatenate(adf2[lst_col].values)})[adf2.columns.tolist()]


adf2['prob'] = adf['concept_vocab_word_tfidf']
adf2['prob'] = adf2['prob'].apply(lambda x:ast.literal_eval(x))
adf2['value'] = adf2['value'].apply(lambda x:ast.literal_eval(x))
adf2.head(50)

adf2 = adf2.explode(['value','prob'])
adf2 = adf2.drop_duplicates(subset=['head','variable','value','prob'], keep='last')

adf2.head(50)

# type(adf2['value'][0])

In [ ]:
adf2.shape

Below code creates a DataFrame adf3 which will contain the triples for the "prerequisite" relation

In [ ]:
adf['prerequisite'] = adf['prerequisite'].fillna('start')
adf['composite'] = adf['composite'].fillna('end')
adf3 = adf[['file_name','prerequisite']]
adf3 = adf3.dropna()
adf3 = adf3.melt('file_name')
#df15 = df15.drop_duplicates(subset=['topics','variable','value'], keep='last')
adf3.rename(columns = {"file_name": "head"},  
          inplace = True) 

adf3 = adf3.drop_duplicates(subset=['head','variable','value'], keep='last')
adf3.head()

Below code creates a DataFrame adf4 which will contain the triples for the "level" relation

In [ ]:
adf4 = adf[['file_name']]
adf4['variable'] = 'level'
adf4['value'] = 'level_1'

adf4['value'] = adf4.apply(lambda row: 'level_1' if row['file_name'][6] == '1' else ('level_2' if row['file_name'][6] == '3' else 'level_3'), axis=1)

adf4.rename(columns = {"file_name": "head"},  
          inplace = True) 
# adf4.loc[adf4['file_name'][6] == '2', 'value'] = '3'
# adf4.loc[adf4['file_name'][6] == '3', 'value'] = '2'
adf4.head()

Below code creates a DataFrame adf5 which will contain the triples for the "concept_vocab_relation" relation (topics - concept_vocab_index - concept_vocab)

In [ ]:
#topics to vacab have to be done separately
tw_df1 = pd.read_csv('/content/drive/MyDrive/IIITB Docs/sem2/EduEmbed/EduEmbedd/data_prepare/tw_df1.csv')
adf5 = tw_df1[['topic_num','cv_index_1']]
adf5 = adf5.dropna()
adf5 = adf5.melt('topic_num')
#df15 = df15.drop_duplicates(subset=['topics','variable','value'], keep='last')
adf5.rename(columns = {"topic_num": "head"},  
          inplace = True) 

adf5 = adf5.drop_duplicates(subset=['head','variable','value'], keep='last')
adf5['head'] = adf5['head'] + 1
adf5['head'] = 'topic_' + adf5['head'].astype(str)
adf5['variable'] = "concept_vocab_index"
adf5.head()

In [ ]:
adf5.shape

Here we are merging all the above created dataframes for different relations and save them as "triples.csv" which will be used for training the KGE models

In [ ]:
fdf = pd.concat([adf1,adf2,adf3,adf4,adf5])


Assigning the default weights to the remainging relations where weight value is not defined


In [ ]:
fdf = fdf.fillna(1)
fdf

In [ ]:
fdf.to_csv('triples.csv')